In [58]:
"""
Agent Processing System
Multi-Agent Financial Analysis - Specialized LLM Agents
Processes AAPL stock query with 4 specialized agents
"""

from openai import OpenAI
from typing import Dict, List, Any
from dataclasses import dataclass
import json
from datetime import datetime
import os

In [59]:
# Configuration - Initialize OpenAI client
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY", "api-key"))


In [60]:
@dataclass
class AgentResponse:
    """Standard response format from all agents"""
    agent_name: str                  # Which agent produced this (e.g., "News Analysis Agent")
    analysis: str                    # The actual text analysis/recommendation
    score: float                     # -1(very negative) to +1 scale (very positive)
    confidence: float                # How confident: 0 (not confident) to 1 (very confident)
    key_factors: List[str]           # Bullet points of important findings
    timestamp: str                   # When the analysis was done

In [61]:
class BaseAgent:
    """Base class for all financial agents"""

    def __init__(self, agent_name: str, model: str = "gpt-4"):
        self.agent_name = agent_name # Stores the agent's name
        self.model = model    # Which AI model to use (default: GPT-4)
        self.memory = []   # Empty list to store conversation history

    def call_llm(self, system_prompt: str, user_message: str) -> str:
        """Call LLM with error handling"""
        try:
            response = client.chat.completions.create(
                model=self.model,
                messages=[
                    {"role": "system", "content": system_prompt}, #Instructions telling the AI how to behave
                    {"role": "user", "content": user_message}  #The actual data/question to analyze
                ],
                temperature=0.3,  # Lower temperature for more consistent analysis
                max_tokens=800     #limits response length
            )
            return response.choices[0].message.content
        except Exception as e:
            print(f"Error in {self.agent_name}: {str(e)}")
            return f"Error processing request: {str(e)}"

    # Add to BaseAgent class
    def add_to_memory(self, interaction):
      """Store conversation history"""
      self.memory.append({
        'timestamp': datetime.now(),
        'input': interaction['input'],
        'output': interaction['output']
        })

    def get_context(self, last_n=5):
      """Retrieve recent context"""
      return self.memory[-last_n:]

    def process(self, data: Dict[str, Any]) -> AgentResponse:
        """Override in each specialized agent"""
        raise NotImplementedError("Each agent must implement process method")

## NewsAnalysisAgent
This agent reads news headlines about a stock, asks GPT-4 "are these headlines good or bad news?", gets back a sentiment score, and packages it in a standardized format for the orchestrator to use.
For example: if someone whats get sentiment from new, this agent is  reading news and telling if it's bullish or bearish.

In [62]:
class NewsAnalysisAgent(BaseAgent):
    """Analyzes financial news sentiment and impact"""

    def __init__(self):
        super().__init__("News Analysis Agent")
        self.system_prompt = """You are a financial news analyst specializing in sentiment analysis.
Analyze news articles about companies and provide:
1. Overall sentiment score (-1 to +1, where -1 is very negative, 0 is neutral, +1 is very positive)
2. Key factors driving the sentiment
3. Potential impact on stock price

Be objective and consider both positive and negative aspects.
Return response in JSON format with keys: sentiment_score, analysis, key_factors, confidence"""

    def process(self, data: Dict[str, Any]) -> AgentResponse:
        """Process news data for sentiment analysis"""
        ticker = data.get('ticker', 'AAPL')
        news_articles = data.get('news', [])

        # Prepare news summary for analysis
        news_summary = "\n".join([
            f"- {article.get('title', '')}: {article.get('description', '')}"
            for article in news_articles[:5]  # Limit to 5 most recent
        ])

        user_message = f"""Analyze the following recent news about {ticker}:

{news_summary}

Provide sentiment analysis and impact assessment."""

        # Get LLM response
        llm_response = self.call_llm(self.system_prompt, user_message)

        # Parse response
        try:
            result = json.loads(llm_response)
            score = result.get('sentiment_score', 0)
            analysis = result.get('analysis', llm_response)
            key_factors = result.get('key_factors', [])
            confidence = result.get('confidence', 0.7)
        except json.JSONDecodeError:
            # Fallback if JSON parsing fails
            score = 0
            analysis = llm_response
            key_factors = ["Unable to parse structured response"]
            confidence = 0.5

        return AgentResponse(
            agent_name=self.agent_name,
            analysis=analysis,
            score=float(score),
            confidence=float(confidence),
            key_factors=key_factors,
            timestamp=datetime.now().isoformat()
        )

## EarningsAnalysisAgent
The Earnings Analysis Agent examines a company's financial performance (revenue, profits, earnings per share) and determines if the fundamentals are strong by comparing actual results against analyst expectations. It sends this financial data to GPT-4, which returns a score (-1 to +1) indicating whether the company's financials suggest it's a good or weak investment.

In [63]:
class EarningsAnalysisAgent(BaseAgent):
    """Analyzes earnings reports and financial statements"""

    def __init__(self):
        super().__init__("Earnings Analysis Agent")
        self.system_prompt = """You are a financial analyst specializing in earnings and fundamental analysis.
Analyze company financial data and provide:
1. Fundamental strength score (-1 to +1, where -1 is very weak, +1 is very strong)
2. Key financial metrics analysis (revenue, earnings, growth)
3. Comparison to expectations
4. Important trends

Return response in JSON format with keys: fundamental_score, analysis, key_factors, confidence"""

    def process(self, data: Dict[str, Any]) -> AgentResponse:
        """Process earnings and financial data"""
        ticker = data.get('ticker', 'UNKNOWN')
        financials = data.get('financials', {})

        # Prepare financial summary
        financial_summary = f"""
Company: {ticker}
Revenue: ${financials.get('revenue', 'N/A')}B
EPS: ${financials.get('eps', 'N/A')}
Revenue Growth: {financials.get('revenue_growth', 'N/A')}%
Profit Margin: {financials.get('profit_margin', 'N/A')}%
Expected Revenue: ${financials.get('expected_revenue', 'N/A')}B
Expected EPS: ${financials.get('expected_eps', 'N/A')}
"""

        user_message = f"""Analyze the following financial data for {ticker}:

{financial_summary}

Assess fundamental strength and growth prospects."""

        # Get LLM response
        llm_response = self.call_llm(self.system_prompt, user_message)

        # Parse response
        try:
            result = json.loads(llm_response)
            score = result.get('fundamental_score', 0)
            analysis = result.get('analysis', llm_response)
            key_factors = result.get('key_factors', [])
            confidence = result.get('confidence', 0.8)
        except json.JSONDecodeError:
            score = 0
            analysis = llm_response
            key_factors = ["Unable to parse structured response"]
            confidence = 0.6

        return AgentResponse(
            agent_name=self.agent_name,
            analysis=analysis,
            score=float(score),
            confidence=float(confidence),
            key_factors=key_factors,
            timestamp=datetime.now().isoformat()
        )

## MarketSignalsAgent
The MarketSignalsAgent performs technical analysis by examining stock price patterns, trading volume, and technical indicators (like moving averages, RSI, MACD) to identify trends and momentum. It sends this technical data to GPT-4, which returns a score (-1 to +1) indicating whether the stock's price action suggests a bullish or bearish trend based on chart patterns and trading signals.

In [64]:
class MarketSignalsAgent(BaseAgent):
    """Performs technical analysis on market data"""

    def __init__(self):
        super().__init__("Market Signals Agent")
        self.system_prompt = """You are a technical analyst specializing in market signals and price patterns.
Analyze technical indicators and provide:
1. Technical strength score (-1 to +1, where -1 is very bearish, +1 is very bullish)
2. Key technical indicators assessment
3. Support and resistance levels
4. Trend analysis

Return response in JSON format with keys: technical_score, analysis, key_factors, confidence"""

    def process(self, data: Dict[str, Any]) -> AgentResponse:
        """Process technical market data"""
        ticker = data.get('ticker', 'UNKNOWN')
        technicals = data.get('technicals', {})

        # Prepare technical summary
        technical_summary = f"""
Ticker: {ticker}
Current Price: ${technicals.get('current_price', 'N/A')}
50-day MA: ${technicals.get('ma_50', 'N/A')}
200-day MA: ${technicals.get('ma_200', 'N/A')}
RSI: {technicals.get('rsi', 'N/A')}
MACD: {technicals.get('macd', 'N/A')}
Volume: {technicals.get('volume', 'N/A')} (Avg: {technicals.get('avg_volume', 'N/A')})
Support: ${technicals.get('support', 'N/A')}
Resistance: ${technicals.get('resistance', 'N/A')}
"""

        user_message = f"""Analyze the following technical data for {ticker}:

{technical_summary}

Assess technical strength and price momentum."""

        # Get LLM response
        llm_response = self.call_llm(self.system_prompt, user_message)

        # Parse response
        try:
            result = json.loads(llm_response)
            score = result.get('technical_score', 0)
            analysis = result.get('analysis', llm_response)
            key_factors = result.get('key_factors', [])
            confidence = result.get('confidence', 0.7)
        except json.JSONDecodeError:
            score = 0
            analysis = llm_response
            key_factors = ["Unable to parse structured response"]
            confidence = 0.5

        return AgentResponse(
            agent_name=self.agent_name,
            analysis=analysis,
            score=float(score),
            confidence=float(confidence),
            key_factors=key_factors,
            timestamp=datetime.now().isoformat()
        )


## RiskAssessmentAgent
The RiskAssessmentAgent evaluates investment risk by analyzing metrics like beta (volatility), Value at Risk, Sharpe ratio, and sector correlation to determine how risky a stock is for a portfolio. It sends these risk metrics to GPT-4, which returns a risk score (0 to 1, where 0 is low risk and 1 is high risk) along with warnings about potential portfolio concentration or volatility issues.

In [65]:
class RiskAssessmentAgent(BaseAgent):
    """Assesses investment risk and portfolio fit"""

    def __init__(self):
        super().__init__("Risk Assessment Agent")
        self.system_prompt = """You are a risk management analyst specializing in portfolio risk assessment.
Analyze risk metrics and provide:
1. Risk level score (0 to 1, where 0 is very low risk, 1 is very high risk)
2. Key risk factors
3. Portfolio diversification implications
4. Risk-adjusted return assessment

Return response in JSON format with keys: risk_score, analysis, key_factors, confidence"""

    def process(self, data: Dict[str, Any]) -> AgentResponse:
        """Process risk metrics"""
        ticker = data.get('ticker', 'UNKNOWN')
        risk_data = data.get('risk_metrics', {})

        # Prepare risk summary
        risk_summary = f"""
Ticker: {ticker}
Beta: {risk_data.get('beta', 'N/A')}
Volatility (30-day): {risk_data.get('volatility', 'N/A')}%
Value at Risk (5%): ${risk_data.get('var_5', 'N/A')}
Sharpe Ratio: {risk_data.get('sharpe_ratio', 'N/A')}
Max Drawdown: {risk_data.get('max_drawdown', 'N/A')}%
Sector Correlation: {risk_data.get('sector_correlation', 'N/A')}
P/E Ratio: {risk_data.get('pe_ratio', 'N/A')}
"""

        user_message = f"""Analyze the following risk metrics for {ticker}:

{risk_summary}

Assess overall investment risk and portfolio implications."""

        # Get LLM response
        llm_response = self.call_llm(self.system_prompt, user_message)

        # Parse response
        try:
            result = json.loads(llm_response)
            score = result.get('risk_score', 0.5)
            analysis = result.get('analysis', llm_response)
            key_factors = result.get('key_factors', [])
            confidence = result.get('confidence', 0.8)
        except json.JSONDecodeError:
            score = 0.5
            analysis = llm_response
            key_factors = ["Unable to parse structured response"]
            confidence = 0.6

        return AgentResponse(
            agent_name=self.agent_name,
            analysis=analysis,
            score=float(score),
            confidence=float(confidence),
            key_factors=key_factors,
            timestamp=datetime.now().isoformat()
        )


## SynthesisAgent
The SynthesisAgent acts as the "decision maker" that takes all the individual agent analyses (news sentiment, earnings strength, technical signals, risk level) and combines them into a single investment recommendation (STRONG BUY, BUY, HOLD, SELL, STRONG SELL). It sends a summary of all agent scores and findings to GPT-4, which weighs the different perspectives and returns a final actionable recommendation with confidence level and supporting reasoning.

In [66]:
class SynthesisAgent(BaseAgent):
    """Combines insights from all agents into final recommendation"""

    def __init__(self):
        # Initialize with base agent functionality
        super().__init__("Research Synthesis Agent")
        # Define instructions for the AI on how to synthesize multiple analyses
        self.system_prompt = """You are a senior investment analyst who synthesizes multiple analyses into actionable recommendations.
Given analyses from news, earnings, technical, and risk agents, provide:
1. Overall investment recommendation (STRONG BUY, BUY, HOLD, SELL, STRONG SELL)
2. Confidence level (0 to 1)
3. Key reasoning
4. Risk considerations
5. Target price range (if applicable)

Return response in JSON format with keys: recommendation, confidence, analysis, key_points, risks"""

    def process(self, agent_responses: List[AgentResponse]) -> AgentResponse:
        """Synthesize all agent responses"""

        # Prepare synthesis input
        # Combine all agent analyses into one formatted summary
        analyses_summary = "\n\n".join([
            f"{resp.agent_name}:\n"
            f"Score: {resp.score}\n"
            f"Analysis: {resp.analysis}\n"
            f"Key Factors: {', '.join(resp.key_factors)}"
            for resp in agent_responses
        ])

        user_message = f"""Synthesize the following analyses into a final investment recommendation:

{analyses_summary}

Provide comprehensive investment recommendation with supporting reasoning."""

        # Get LLM response
        llm_response = self.call_llm(self.system_prompt, user_message)

        # Parse response
        # Expected format: {"recommendation": "BUY", "confidence": 0.8, "analysis": "...", ...}
        try:
            result = json.loads(llm_response)
            # Extract recommendation (e.g., "BUY", "HOLD", "SELL")
            recommendation = result.get('recommendation', 'HOLD')
            # Extract detailed analysis text
            analysis = result.get('analysis', llm_response)

            key_factors = result.get('key_points', [])
            # Extract confidence level (0 to 1)
            confidence = result.get('confidence', 0.7)

            # Convert recommendation to score
            rec_to_score = {
                'STRONG BUY': 1.0,
                'BUY': 0.6,
                'HOLD': 0.0,
                'SELL': -0.6,
                'STRONG SELL': -1.0
            }
            score = rec_to_score.get(recommendation, 0.0)

        except json.JSONDecodeError:
            score = 0
            analysis = llm_response # Use raw response
            key_factors = ["Unable to parse structured response"]
            confidence = 0.6 # Lower confidence due to parsing failure

        # Return standardized AgentResponse object
        return AgentResponse(
            agent_name=self.agent_name,
            analysis=analysis,
            score=float(score),
            confidence=float(confidence),
            key_factors=key_factors,
            timestamp=datetime.now().isoformat()
        )


## CritiqueAgent

The CritiqueAgent acts as a "quality control checker" that reviews the final investment recommendation to catch mistakes, biases, or missing information before presenting it to the user. It examines the SynthesisAgent's recommendation, asks GPT-4 to identify logical flaws or gaps in reasoning, and can adjust the confidence level downward if it finds issues (like  "didn't consider macroeconomic factors"), ensuring the final output is reliable and well-reasoned.


In [70]:
class CritiqueAgent(BaseAgent):
    """Reviews and validates analysis quality"""

    def __init__(self):
        super().__init__("Critique & Validation Agent")
        self.system_prompt = """You are critique analyst who reviews investment recommendations for biases, logical errors, and completeness.
Review the synthesis and identify:
1. Logical inconsistencies
2. Potential biases
3. Missing considerations
4. Data quality issues
5. Confidence adjustment recommendation

Return response in JSON format with keys: quality_score, issues_found, suggestions, adjusted_confidence"""

    def process(self, synthesis_response: AgentResponse) -> AgentResponse:
        """Critique the synthesis"""

        user_message = f"""Review this investment analysis for quality and completeness:

Recommendation: {synthesis_response.analysis}
Confidence: {synthesis_response.confidence}
Key Factors: {', '.join(synthesis_response.key_factors)}

Identify any issues, biases, or missing elements."""

        # Get LLM response
        llm_response = self.call_llm(self.system_prompt, user_message)

        # Parse response
        try:
            result = json.loads(llm_response)
            quality_score = result.get('quality_score', 0.7)
            issues = result.get('issues_found', [])
            suggestions = result.get('suggestions', [])
            adjusted_confidence = result.get('adjusted_confidence', synthesis_response.confidence)

            analysis = f"Quality Score: {quality_score}\n"
            if issues:
                analysis += f"Issues Found: {', '.join(issues)}\n"
            if suggestions:
                analysis += f"Suggestions: {', '.join(suggestions)}"

            key_factors = issues if issues else ["No major issues found"]

        except json.JSONDecodeError:
            quality_score = 0.7
            analysis = llm_response
            adjusted_confidence = synthesis_response.confidence
            key_factors = ["No major issues found"]

        return AgentResponse(
            agent_name=self.agent_name,
            analysis=analysis,
            score=float(quality_score),
            confidence=float(adjusted_confidence),
            key_factors=key_factors,
            timestamp=datetime.now().isoformat()
        )


In [69]:

# ============================================================
# MOCK TESTING (for testing without API)
# ============================================================

def mock_call_llm(self, system_prompt: str, user_message: str) -> str:
    """Mock LLM - no API calls"""
    if "news" in self.agent_name.lower():
        return json.dumps({
            "sentiment_score": 0.6,
            "analysis": "Mock: Positive sentiment",
            "key_factors": ["Strong earnings", "Product concerns"],
            "confidence": 0.75
        })
    elif "earnings" in self.agent_name.lower():
        return json.dumps({
            "fundamental_score": 0.8,
            "analysis": "Mock: Strong fundamentals",
            "key_factors": ["Revenue beat", "EPS growth"],
            "confidence": 0.85
        })
    return json.dumps({"score": 0.5, "analysis": "Mock", "key_factors": [], "confidence": 0.7})

# Patch BaseAgent to use mock
BaseAgent.call_llm = mock_call_llm
print("Mock mode enabled\n")

test_data = {'ticker': 'AAPL', 'news': [{'title': 'Test', 'description': 'Test'}]}
agent = NewsAnalysisAgent()
result = agent.process(test_data)

print(f"Score: {result.score}")
print(f"Confidence: {result.confidence}")
print(f"Analysis : {result.analysis}")
print(f"Key Factors : {result.key_factors}")

Mock mode enabled

Score: 0.6
Confidence: 0.75
Analysis : Mock: Positive sentiment
Key Factors : ['Strong earnings', 'Product concerns']
